In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
%matplotlib inline

In [2]:
sns.set_style('whitegrid')

**Importing Training Data**

In [3]:
beneficiary = pd.read_csv('datasets/Train_Beneficiarydata-1542865627584.csv')
inPatient = pd.read_csv('datasets/Train_Inpatientdata-1542865627584.csv')
outPatient = pd.read_csv('datasets/Train_Outpatientdata-1542865627584.csv')

**Importing Testing Data**

In [4]:
beneficiary_test = pd.read_csv('datasets/Test_Beneficiarydata-1542969243754.csv')
inPatient_test = pd.read_csv('datasets/Test_Inpatientdata-1542969243754.csv')
outPatient_test = pd.read_csv('datasets/Test_Outpatientdata-1542969243754.csv')

In [5]:
outPatient.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,11-10-2009,11-10-2009,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,12-02-2009,12-02-2009,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,27-06-2009,27-06-2009,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,06-01-2009,06-01-2009,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,22-01-2009,22-01-2009,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [6]:
inPatient.drop(['AdmissionDt','DischargeDt','ClaimID'], axis=1, inplace=True)

#Testing data cleaning
inPatient_test.drop(['AdmissionDt','DischargeDt','ClaimID'], axis=1, inplace=True)

In [7]:
outPatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  517737 non-null  object 
 1   ClaimID                 517737 non-null  object 
 2   ClaimStartDt            517737 non-null  object 
 3   ClaimEndDt              517737 non-null  object 
 4   Provider                517737 non-null  object 
 5   InscClaimAmtReimbursed  517737 non-null  int64  
 6   AttendingPhysician      516341 non-null  object 
 7   OperatingPhysician      90617 non-null   object 
 8   OtherPhysician          195046 non-null  object 
 9   ClmDiagnosisCode_1      507284 non-null  object 
 10  ClmDiagnosisCode_2      322357 non-null  object 
 11  ClmDiagnosisCode_3      203257 non-null  object 
 12  ClmDiagnosisCode_4      125596 non-null  object 
 13  ClmDiagnosisCode_5      74344 non-null   object 
 14  ClmDiagnosisCode_6  

In [8]:
inPatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BeneID                  40474 non-null  object 
 1   ClaimStartDt            40474 non-null  object 
 2   ClaimEndDt              40474 non-null  object 
 3   Provider                40474 non-null  object 
 4   InscClaimAmtReimbursed  40474 non-null  int64  
 5   AttendingPhysician      40362 non-null  object 
 6   OperatingPhysician      23830 non-null  object 
 7   OtherPhysician          4690 non-null   object 
 8   ClmAdmitDiagnosisCode   40474 non-null  object 
 9   DeductibleAmtPaid       39575 non-null  float64
 10  DiagnosisGroupCode      40474 non-null  object 
 11  ClmDiagnosisCode_1      40474 non-null  object 
 12  ClmDiagnosisCode_2      40248 non-null  object 
 13  ClmDiagnosisCode_3      39798 non-null  object 
 14  ClmDiagnosisCode_4      38940 non-null

In [9]:
#sns.jointplot(x='BeneID', y='InscClaimAmtReimbursed', data = inPatient)

In [10]:
outPatient['DiagnosisGroupCode'] = pd.NA
outPatient['DiagnosisGroupCode'] = outPatient['DiagnosisGroupCode'].fillna(-1)

#test data
outPatient_test['DiagnosisGroupCode'] = pd.NA
outPatient_test['DiagnosisGroupCode'] = outPatient_test['DiagnosisGroupCode'].fillna(-1)

In [11]:
patient = pd.concat([inPatient,outPatient])

#test data
patient_test = pd.concat([inPatient_test,outPatient_test])

In [12]:
numeric_values = pd.to_numeric(patient['DiagnosisGroupCode'], errors='coerce',downcast="float")


patient['DiagnosisGroupCode'] = pd.to_numeric(patient['DiagnosisGroupCode'], errors='coerce',downcast="float")
patient_test['DiagnosisGroupCode'] = pd.to_numeric(patient_test['DiagnosisGroupCode'], errors='coerce',downcast="float")

# Count the number of NaN values
non_integer_count = numeric_values.isna().sum()

print("Number of rows not containing integers in 'DiagnosisGroupCode' column in training dataset:", non_integer_count)

Number of rows not containing integers in 'DiagnosisGroupCode' column in training dataset: 113


In [13]:
patient['ClmAdmitDiagnosisCode'] = patient['ClmAdmitDiagnosisCode'].fillna(-1)


numeric_values = pd.to_numeric(patient['ClmAdmitDiagnosisCode'], errors='coerce',downcast="float")

patient['ClmAdmitDiagnosisCode'] = pd.to_numeric(patient['ClmAdmitDiagnosisCode'], errors='coerce',downcast="float")
patient_test['ClmAdmitDiagnosisCode'] = pd.to_numeric(patient_test['ClmAdmitDiagnosisCode'], errors='coerce',downcast="float")

#Count the number of NaN values
non_integer_count = numeric_values.isna().sum()

print("Number of rows not containing integers in 'ClmAdmitDiagnosisCode' column in training dataset:", non_integer_count)

Number of rows not containing integers in 'ClmAdmitDiagnosisCode' column in training dataset: 19985


In [14]:
patient = patient.dropna(subset=['ClmAdmitDiagnosisCode','DiagnosisGroupCode'])
patient_test = patient_test.dropna(subset=['ClmAdmitDiagnosisCode','DiagnosisGroupCode'])

In [15]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538118 entries, 0 to 517736
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  538118 non-null  object 
 1   ClaimStartDt            538118 non-null  object 
 2   ClaimEndDt              538118 non-null  object 
 3   Provider                538118 non-null  object 
 4   InscClaimAmtReimbursed  538118 non-null  int64  
 5   AttendingPhysician      536728 non-null  object 
 6   OperatingPhysician      111146 non-null  object 
 7   OtherPhysician          197480 non-null  object 
 8   ClmAdmitDiagnosisCode   538118 non-null  float32
 9   DeductibleAmtPaid       537251 non-null  float64
 10  DiagnosisGroupCode      538118 non-null  float32
 11  ClmDiagnosisCode_1      528296 non-null  object 
 12  ClmDiagnosisCode_2      350366 non-null  object 
 13  ClmDiagnosisCode_3      235826 non-null  object 
 14  ClmDiagnosisCode_4      1

In [16]:
gr_users = patient.groupby("BeneID")
gr_users.size()

BeneID
BENE100000     2
BENE100001     9
BENE100002    13
BENE100003     1
BENE100004     6
              ..
BENE99994      7
BENE99995      1
BENE99997      7
BENE99998      3
BENE99999      5
Length: 137382, dtype: int64

In [17]:
# Convert 'ClaimEndDt' and 'ClaimStartDt' to datetime objects with the correct date format
patient['ClaimEndDt'] = pd.to_datetime(patient['ClaimEndDt'], format='%d-%m-%Y')
patient['ClaimStartDt'] = pd.to_datetime(patient['ClaimStartDt'], format='%d-%m-%Y')


#Testing dataset
patient_test['ClaimEndDt'] = pd.to_datetime(patient_test['ClaimEndDt'], format='%d-%m-%Y')
patient_test['ClaimStartDt'] = pd.to_datetime(patient_test['ClaimStartDt'], format='%d-%m-%Y')

In [18]:
patient['Days admitted'] = (patient['ClaimEndDt'] - patient['ClaimStartDt']).dt.days
patient.drop(['ClaimStartDt', 'ClaimEndDt'], axis=1, inplace=True)
#patient['Days admitted'] = patient['Days admitted'].astype(int)


#Testing dataset
patient_test['Days admitted'] = (patient_test['ClaimEndDt'] - patient_test['ClaimStartDt']).dt.days
patient_test.drop(['ClaimStartDt', 'ClaimEndDt'], axis=1, inplace=True)
patient.head()

,BeneID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimID,Days admitted
0,BENE11001,PRV55912,26000,PHY390922,NaN,NaN,7866.0,1068.0,201.0,1970,...,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
1,BENE11001,PRV55907,5000,PHY318495,PHY318495,NaN,6186.0,1068.0,750.0,6186,...,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,NaN,2
2,BENE11001,PRV56046,5000,PHY372395,NaN,PHY324689,29590.0,1068.0,883.0,29623,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,BENE11011,PRV52405,5000,PHY369659,PHY392961,PHY349768,431.0,1068.0,67.0,43491,...,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,NaN,8
4,BENE11014,PRV56614,10000,PHY379376,PHY398258,NaN,78321.0,1068.0,975.0,42,...,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,NaN,17


In [19]:
patient.columns

Index(['BeneID', 'Provider', 'InscClaimAmtReimbursed', 'AttendingPhysician',
       'OperatingPhysician', 'OtherPhysician', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DiagnosisGroupCode', 'ClmDiagnosisCode_1',
       'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4',
       'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7',
       'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10',
       'ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3',
       'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6',
       'ClaimID', 'Days admitted'],
      dtype='object')

In [20]:
# Define weights for each diagnosis code
weights = {
    'ClmDiagnosisCode_1': 10,
    'ClmDiagnosisCode_2': 9,
    'ClmDiagnosisCode_3': 8,
    'ClmDiagnosisCode_4': 7,
    'ClmDiagnosisCode_5': 6,
    'ClmDiagnosisCode_6': 5,
    'ClmDiagnosisCode_7': 4,
    'ClmDiagnosisCode_8': 3,
    'ClmDiagnosisCode_9': 2,
    'ClmDiagnosisCode_10': 1
}

cols = ['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10']
# Calculate Diagnosis Score based on weights
def calculate_diagnosis_score(row):
    score = 0
    for column, weight in weights.items():
        if not pd.isnull(row[column]):
            score += weight
    return score

# Add DiagnosisScore column to the DataFrame
patient['DiagnosisScore'] = patient.apply(calculate_diagnosis_score, axis=1)
patient.drop(cols,axis=1,inplace=True)


#Testing dataset
patient_test['DiagnosisScore'] = patient_test.apply(calculate_diagnosis_score, axis=1)
patient_test.drop(cols,axis=1,inplace=True)
# Display the DataFrame with the new DiagnosisScore column
patient.tail(10)

,BeneID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimID,Days admitted,DiagnosisScore
517727,BENE159198,PRV53672,70,PHY317739,PHY317739,PHY423886,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM255268,0,10
517728,BENE159198,PRV53699,50,PHY380182,NaN,NaN,71946.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM275604,20,34
517729,BENE159198,PRV53670,0,PHY329971,NaN,NaN,29570.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM310720,20,19
517730,BENE159198,PRV53676,80,PHY361063,NaN,NaN,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM347778,0,19
517731,BENE159198,PRV53699,100,PHY380182,NaN,PHY385752,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM400395,0,10
517732,BENE159198,PRV53699,800,PHY364188,PHY364188,PHY385752,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM510792,0,27
517733,BENE159198,PRV53702,400,PHY423019,PHY332284,NaN,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM551294,0,27
517734,BENE159198,PRV53676,60,PHY361063,NaN,NaN,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM596444,0,19
517735,BENE159198,PRV53689,70,PHY403198,NaN,PHY419379,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM636992,0,0
517736,BENE159198,PRV53689,80,PHY419379,NaN,PHY419379,-1.0,0.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,CLM686139,1,52


In [21]:
# Define weights for each Procedure code
weights = {
    'ClmProcedureCode_1': 6,
    'ClmProcedureCode_2': 5,
    'ClmProcedureCode_3': 4,
    'ClmProcedureCode_4': 3,
    'ClmProcedureCode_5': 2,
    'ClmProcedureCode_6': 1,
}

cols = ['ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6']

# Calculate Procedure Score based on weights
def calculate_procedure_score(row):
    score = 0
    for column, weight in weights.items():
        if not pd.isnull(row[column]):
            score += weight
    return score

# Add ProcedureScore column to the DataFrame
patient['ProcedureScore'] = patient.apply(calculate_procedure_score, axis=1)
patient.drop(cols,axis=1,inplace=True)


#Testing dataset
patient_test['ProcedureScore'] = patient_test.apply(calculate_procedure_score, axis=1)
patient_test.drop(cols,axis=1,inplace=True)

# Display the DataFrame with the new ProcedureScore column
patient.head(10)

,BeneID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClaimID,Days admitted,DiagnosisScore,ProcedureScore
0,BENE11001,PRV55912,26000,PHY390922,NaN,NaN,7866.0,1068.0,201.0,NaN,6,54,0
1,BENE11001,PRV55907,5000,PHY318495,PHY318495,NaN,6186.0,1068.0,750.0,NaN,2,27,6
2,BENE11001,PRV56046,5000,PHY372395,NaN,PHY324689,29590.0,1068.0,883.0,NaN,3,45,0
3,BENE11011,PRV52405,5000,PHY369659,PHY392961,PHY349768,431.0,1068.0,67.0,NaN,8,54,6
4,BENE11014,PRV56614,10000,PHY379376,PHY398258,NaN,78321.0,1068.0,975.0,NaN,17,54,6
5,BENE11017,PRV54986,8000,PHY402711,PHY402711,PHY402711,1749.0,1068.0,597.0,NaN,6,54,6
6,BENE11018,PRV54090,8000,PHY412314,PHY347494,NaN,5699.0,1068.0,390.0,NaN,5,54,6
7,BENE11028,PRV51148,6000,PHY346286,PHY405514,NaN,78605.0,1068.0,379.0,NaN,4,54,6
8,BENE11031,PRV55839,7000,PHY385030,NaN,NaN,2859.0,1068.0,294.0,NaN,3,45,0
9,BENE11034,PRV55215,29000,PHY355604,PHY415867,NaN,41401.0,1068.0,262.0,NaN,7,54,11


In [22]:
# Define weights for each Physician code
weights = {
    'AttendingPhysician': 15,
    'OperatingPhysician': 10,
    'OtherPhysician': 5
}
cols = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']

# Calculate Physician Score based on weights
def calculate_physician_score(row):
    score = 0
    for column, weight in weights.items():
        if not pd.isnull(row[column]):
            score += weight
    return score

# Add PhysicianScore column to the DataFrame
patient['PhysicianScore'] = patient.apply(calculate_physician_score, axis=1)
patient.drop(cols,axis=1,inplace=True)


#Testing dataset
patient_test['PhysicianScore'] = patient_test.apply(calculate_physician_score, axis=1)
patient_test.drop(cols,axis=1,inplace=True)
# Display the DataFrame with the new ProcedureScore column
patient.tail(10)

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClaimID,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore
517727,BENE159198,PRV53672,70,-1.0,0.0,-1.0,CLM255268,0,10,0,30
517728,BENE159198,PRV53699,50,71946.0,0.0,-1.0,CLM275604,20,34,0,15
517729,BENE159198,PRV53670,0,29570.0,0.0,-1.0,CLM310720,20,19,0,15
517730,BENE159198,PRV53676,80,-1.0,0.0,-1.0,CLM347778,0,19,0,15
517731,BENE159198,PRV53699,100,-1.0,0.0,-1.0,CLM400395,0,10,0,20
517732,BENE159198,PRV53699,800,-1.0,0.0,-1.0,CLM510792,0,27,0,30
517733,BENE159198,PRV53702,400,-1.0,0.0,-1.0,CLM551294,0,27,0,25
517734,BENE159198,PRV53676,60,-1.0,0.0,-1.0,CLM596444,0,19,0,15
517735,BENE159198,PRV53689,70,-1.0,0.0,-1.0,CLM636992,0,0,0,20
517736,BENE159198,PRV53689,80,-1.0,0.0,-1.0,CLM686139,1,52,0,20


In [23]:
patient.head()

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClaimID,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore
0,BENE11001,PRV55912,26000,7866.0,1068.0,201.0,NaN,6,54,0,15
1,BENE11001,PRV55907,5000,6186.0,1068.0,750.0,NaN,2,27,6,25
2,BENE11001,PRV56046,5000,29590.0,1068.0,883.0,NaN,3,45,0,20
3,BENE11011,PRV52405,5000,431.0,1068.0,67.0,NaN,8,54,6,30
4,BENE11014,PRV56614,10000,78321.0,1068.0,975.0,NaN,17,54,6,25


In [24]:
patient['NaN count'] = patient.isna().sum(axis=1)

#Testing dataset
patient_test['NaN count'] = patient_test.isna().sum(axis=1)

#Changing setting to all the columns for checking
pd.set_option('display.max_columns', None)
patient.info()

<class 'pandas.core.frame.DataFrame'>
Index: 538118 entries, 0 to 517736
Data columns (total 12 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   BeneID                  538118 non-null  object 
 1   Provider                538118 non-null  object 
 2   InscClaimAmtReimbursed  538118 non-null  int64  
 3   ClmAdmitDiagnosisCode   538118 non-null  float32
 4   DeductibleAmtPaid       537251 non-null  float64
 5   DiagnosisGroupCode      538118 non-null  float32
 6   ClaimID                 498954 non-null  object 
 7   Days admitted           538118 non-null  int64  
 8   DiagnosisScore          538118 non-null  int64  
 9   ProcedureScore          538118 non-null  int64  
 10  PhysicianScore          538118 non-null  int64  
 11  NaN count               538118 non-null  int64  
dtypes: float32(2), float64(1), int64(6), object(3)
memory usage: 49.3+ MB


In [25]:
patient.columns

Index(['BeneID', 'Provider', 'InscClaimAmtReimbursed', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DiagnosisGroupCode', 'ClaimID', 'Days admitted',
       'DiagnosisScore', 'ProcedureScore', 'PhysicianScore', 'NaN count'],
      dtype='object')

In [26]:
#n = 50000  # Example: removing top 100000 rows
#
#patient['Total Score'] = patient['DiagnosisScore']+patient['NaN count']+patient['PhysicianScore']+patient['ProcedureScore']
#patient = patient.sort_values(by='Total Score')
## Remove the top 'n' rows
#cleaned_df = patient.iloc[n:]
#
#
#
##Testing dataset
#patient_test['Total Score'] = patient_test['DiagnosisScore']+patient_test['NaN count']+patient_test['PhysicianScore']+patient_test['ProcedureScore']
##patient_test = patient_test.sort_values(by='Total Score')
#
#
## Display the cleaned DataFrame
##print(cleaned_df)
#print(patient['NaN count'].mean())
#print(cleaned_df['NaN count'].mean())
#patient = patient.iloc[n:]
##patient_test = patient_test.iloc[n:]

In [27]:
patient['BeneID'] = patient['BeneID'].str.replace('^BENE', '', regex=True)
patient['Provider'] = patient['Provider'].astype(str).str.replace('^PRV', '', regex=True)
#patient['ClaimID'] = patient['ClaimID'].str.replace('^CLM', '', regex=True)


#Testing dataset
patient_test['BeneID'] = patient_test['BeneID'].str.replace('^BENE', '', regex=True)
patient_test['Provider'] = patient_test['Provider'].astype(str).str.replace('^PRV', '', regex=True)
#patient_test['ClaimID'] = patient_test['ClaimID'].str.replace('^CLM', '', regex=True)


patient.head()

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClaimID,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore,NaN count
0,11001,55912,26000,7866.0,1068.0,201.0,NaN,6,54,0,15,1
1,11001,55907,5000,6186.0,1068.0,750.0,NaN,2,27,6,25,1
2,11001,56046,5000,29590.0,1068.0,883.0,NaN,3,45,0,20,1
3,11011,52405,5000,431.0,1068.0,67.0,NaN,8,54,6,30,1
4,11014,56614,10000,78321.0,1068.0,975.0,NaN,17,54,6,25,1


In [28]:
#def enumerate_values(df, selected_columns):
#    value_to_int = {}
#    for col in selected_columns:
#        unique_values = df[col].unique()
#        for value in unique_values:
#            if value not in value_to_int:
#                value_to_int[value] = len(value_to_int) + 1
#
#    for col in selected_columns:
#        df[col] = df[col].map(value_to_int)
#
#    return df

In [29]:
#selected_columns = ['ClmAdmitDiagnosisCode', 'DeductibleAmtPaid', 'DiagnosisGroupCode']
patient.drop(['DeductibleAmtPaid','ClaimID'],axis=1,inplace=True)
patient_test.drop(['DeductibleAmtPaid','ClaimID'],axis=1,inplace=True)
#patient = enumerate_values(patient, selected_columns)

#Testing dataset
#patient_test = enumerate_values(patient_test, selected_columns)

patient.head()

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DiagnosisGroupCode,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore,NaN count
0,11001,55912,26000,7866.0,201.0,6,54,0,15,1
1,11001,55907,5000,6186.0,750.0,2,27,6,25,1
2,11001,56046,5000,29590.0,883.0,3,45,0,20,1
3,11011,52405,5000,431.0,67.0,8,54,6,30,1
4,11014,56614,10000,78321.0,975.0,17,54,6,25,1


In [30]:
selected_columns = ['BeneID','InscClaimAmtReimbursed','ClmAdmitDiagnosisCode','Provider',
      'Days admitted', 'NaN count', 'DiagnosisScore','ProcedureScore','PhysicianScore','Total Score']
corr = patient.corr()
corr

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DiagnosisGroupCode,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore,NaN count
BeneID,1.000000,-0.000435,-0.000581,0.001221,0.000155,0.002694,0.002354,-0.000090,0.002024,0.000256
Provider,-0.000435,1.000000,-0.011628,-0.008430,-0.016193,-0.005122,-0.009017,-0.013178,-0.003733,-0.017846
InscClaimAmtReimbursed,-0.000581,-0.011628,1.000000,0.279050,0.558359,0.251168,0.354463,0.642231,0.175243,0.655173
ClmAdmitDiagnosisCode,0.001221,-0.008430,0.279050,1.000000,0.344000,0.051901,0.236899,0.304488,0.042899,0.412207
DiagnosisGroupCode,0.000155,-0.016193,0.558359,0.344000,1.000000,0.197382,0.417789,0.596615,0.133632,0.838225
Days admitted,0.002694,-0.005122,0.251168,0.051901,0.197382,1.000000,0.199738,0.193763,0.029432,0.218131
DiagnosisScore,0.002354,-0.009017,0.354463,0.236899,0.417789,0.199738,1.000000,0.357156,0.215654,0.494135
ProcedureScore,-0.000090,-0.013178,0.642231,0.304488,0.596615,0.193763,0.357156,1.000000,0.278149,0.694206
PhysicianScore,0.002024,-0.003733,0.175243,0.042899,0.133632,0.029432,0.215654,0.278149,1.000000,0.149155
NaN count,0.000256,-0.017846,0.655173,0.412207,0.838225,0.218131,0.494135,0.694206,0.149155,1.000000


In [31]:
#sns.jointplot(x='BeneID', y='InscClaimAmtReimbursed', data = outPatient)

In [32]:
beneficiary.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,01-01-1943,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,01-09-1936,NaN,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,01-08-1936,NaN,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,01-07-1922,NaN,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,01-09-1935,NaN,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200


In [33]:
drop_cols = ['DOB','DOD','Race','State','County','NoOfMonths_PartACov','NoOfMonths_PartBCov']
beneficiary = beneficiary.drop(columns = drop_cols, axis = 1)

#Testing dataset
drop_cols = ['DOB','DOD','Race','State','County','NoOfMonths_PartACov','NoOfMonths_PartBCov']
beneficiary_test = beneficiary_test.drop(columns = drop_cols, axis = 1)

beneficiary

,BeneID,Gender,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1,0,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,2,0,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1,0,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1,0,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1,0,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138551,BENE159194,1,0,1,2,2,2,2,2,2,2,2,2,2,0,0,430,460
138552,BENE159195,2,0,1,2,2,2,2,2,1,2,2,2,2,0,0,880,100
138553,BENE159196,2,0,2,1,1,1,2,1,1,1,2,2,2,2000,1068,3240,1390
138554,BENE159197,1,0,1,1,2,2,2,2,2,1,2,2,2,0,0,2650,10


In [34]:
beneficiary.columns

Index(['BeneID', 'Gender', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
       'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
       'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt'],
      dtype='object')

In [35]:
beneficiary['BeneID'] = beneficiary['BeneID'].str.replace('^BENE', '', regex=True)
beneficiary_test['BeneID'] = beneficiary_test['BeneID'].str.replace('^BENE', '', regex=True)

In [36]:
# List of columns to consider for disease score calculation
columns_to_score = ['ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
                    'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
                    'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
                    'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']

# Create a new column 'DiseaseScore' and initialize it to 0
beneficiary['DiseaseScore'] = 0

# Iterate row-wise to calculate the disease score
for index, row in beneficiary.iterrows():
    score = 0
    for column in columns_to_score:
        if row[column] == 1:
            score += 1
    beneficiary.at[index, 'DiseaseScore'] = score


beneficiary.drop(columns_to_score,axis=1,inplace=True)


#Testing dataset
beneficiary_test['DiseaseScore'] = 0

# Iterate row-wise to calculate the disease score
for index, row in beneficiary_test.iterrows():
    score = 0
    for column in columns_to_score:
        if row[column] == 1:
            score += 1
    beneficiary_test.at[index, 'DiseaseScore'] = score


beneficiary_test.drop(columns_to_score,axis=1,inplace=True)

beneficiary.head()

,BeneID,Gender,RenalDiseaseIndicator,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,DiseaseScore
0,11001,1,0,36000,3204,60,70,7
1,11002,2,0,0,0,30,50,0
2,11003,1,0,0,0,90,40,2
3,11004,1,0,0,0,1810,760,6
4,11005,1,0,0,0,1790,1200,2


In [37]:
beneficiary

,BeneID,Gender,RenalDiseaseIndicator,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,DiseaseScore
0,11001,1,0,36000,3204,60,70,7
1,11002,2,0,0,0,30,50,0
2,11003,1,0,0,0,90,40,2
3,11004,1,0,0,0,1810,760,6
4,11005,1,0,0,0,1790,1200,2
...,...,...,...,...,...,...,...,...
138551,159194,1,0,0,0,430,460,1
138552,159195,2,0,0,0,880,100,2
138553,159196,2,0,2000,1068,3240,1390,6
138554,159197,1,0,0,0,2650,10,3


In [38]:
beneficiary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   BeneID                    138556 non-null  object
 1   Gender                    138556 non-null  int64 
 2   RenalDiseaseIndicator     138556 non-null  object
 3   IPAnnualReimbursementAmt  138556 non-null  int64 
 4   IPAnnualDeductibleAmt     138556 non-null  int64 
 5   OPAnnualReimbursementAmt  138556 non-null  int64 
 6   OPAnnualDeductibleAmt     138556 non-null  int64 
 7   DiseaseScore              138556 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 8.5+ MB


In [39]:
print(beneficiary['RenalDiseaseIndicator'].value_counts())
beneficiary['RenalDiseaseIndicator'] = beneficiary['RenalDiseaseIndicator'].map({'Y': 1, '0': 0})

#Testing dataset
beneficiary_test['RenalDiseaseIndicator'] = beneficiary_test['RenalDiseaseIndicator'].map({'Y': 1, '0': 0})

RenalDiseaseIndicator
0    118978
Y     19578
Name: count, dtype: int64


In [40]:
#drop_cols = ['IPAnnualReimbursementAmt','IPAnnualDeductibleAmt','OPAnnualReimbursementAmt','OPAnnualDeductibleAmt']

#beneficiary = beneficiary.drop(columns = drop_cols, axis = 1)
#beneficiary_test = beneficiary_test.drop(columns = drop_cols, axis = 1)

In [41]:
ben_corr = beneficiary.corr()
ben_corr

,BeneID,Gender,RenalDiseaseIndicator,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,DiseaseScore
BeneID,1.000000,0.003153,0.002831,-0.000071,-0.000224,0.005775,0.003650,0.002047
Gender,0.003153,1.000000,0.006836,0.001782,-0.000235,0.007061,0.007625,0.023286
RenalDiseaseIndicator,0.002831,0.006836,1.000000,0.156877,0.148150,0.247980,0.252271,0.301911
IPAnnualReimbursementAmt,-0.000071,0.001782,0.156877,1.000000,0.627806,0.147001,0.124757,0.298400
IPAnnualDeductibleAmt,-0.000224,-0.000235,0.148150,0.627806,1.000000,0.110741,0.110502,0.327539
OPAnnualReimbursementAmt,0.005775,0.007061,0.247980,0.147001,0.110741,1.000000,0.754776,0.226196
OPAnnualDeductibleAmt,0.003650,0.007625,0.252271,0.124757,0.110502,0.754776,1.000000,0.242748
DiseaseScore,0.002047,0.023286,0.301911,0.298400,0.327539,0.226196,0.242748,1.000000


In [42]:
merged_df = pd.merge(patient, beneficiary, on='BeneID', how='left')

#Testing dataset
merged_df_test = pd.merge(patient_test, beneficiary_test, on='BeneID', how='left')

In [43]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538118 entries, 0 to 538117
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   BeneID                    538118 non-null  object 
 1   Provider                  538118 non-null  object 
 2   InscClaimAmtReimbursed    538118 non-null  int64  
 3   ClmAdmitDiagnosisCode     538118 non-null  float32
 4   DiagnosisGroupCode        538118 non-null  float32
 5   Days admitted             538118 non-null  int64  
 6   DiagnosisScore            538118 non-null  int64  
 7   ProcedureScore            538118 non-null  int64  
 8   PhysicianScore            538118 non-null  int64  
 9   NaN count                 538118 non-null  int64  
 10  Gender                    538118 non-null  int64  
 11  RenalDiseaseIndicator     538118 non-null  int64  
 12  IPAnnualReimbursementAmt  538118 non-null  int64  
 13  IPAnnualDeductibleAmt     538118 non-null  i

In [66]:
merged_df.head(1000)

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DiagnosisGroupCode,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore,NaN count,Gender,RenalDiseaseIndicator,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,DiseaseScore
0,11001,55912,26000,7866,201,6,54,0,15,1,1,0,36000,3204,60,70,7
1,11001,55907,5000,6186,750,2,27,6,25,1,1,0,36000,3204,60,70,7
2,11001,56046,5000,29590,883,3,45,0,20,1,1,0,36000,3204,60,70,7
3,11011,52405,5000,431,67,8,54,6,30,1,2,0,5000,1068,250,320,6
4,11014,56614,10000,78321,975,17,54,6,25,1,2,1,21260,2136,120,100,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14649,57414,54000,78650,234,11,54,6,25,1,1,0,66440,2136,40,0,5
996,14656,54765,16000,78097,39,3,34,0,15,1,2,0,16000,1068,1810,1340,5
997,14658,57299,14000,4423,239,2,52,6,25,1,1,0,14200,1068,1100,280,4
998,14659,57031,10000,57420,445,2,54,0,20,1,2,0,10100,1068,690,330,1


In [45]:
ncorr = merged_df.corr()
ncorr

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DiagnosisGroupCode,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore,NaN count,Gender,RenalDiseaseIndicator,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,DiseaseScore
BeneID,1.000000,-0.000435,-0.000581,0.001221,0.000155,0.002694,0.002354,-0.000090,0.002024,0.000256,0.001393,0.004922,-0.001066,-0.000296,0.010718,0.008163,0.002141
Provider,-0.000435,1.000000,-0.011628,-0.008430,-0.016193,-0.005122,-0.009017,-0.013178,-0.003733,-0.017846,0.006047,0.001950,-0.009574,-0.013062,-0.004277,-0.006280,-0.003708
InscClaimAmtReimbursed,-0.000581,-0.011628,1.000000,0.279050,0.558359,0.251168,0.354463,0.642231,0.175243,0.655173,-0.001805,0.060811,0.383371,0.224641,0.066564,0.032133,0.092002
ClmAdmitDiagnosisCode,0.001221,-0.008430,0.279050,1.000000,0.344000,0.051901,0.236899,0.304488,0.042899,0.412207,-0.000896,0.028558,0.134413,0.145173,-0.015894,-0.014766,0.092864
DiagnosisGroupCode,0.000155,-0.016193,0.558359,0.344000,1.000000,0.197382,0.417789,0.596615,0.133632,0.838225,-0.000846,0.039226,0.248186,0.269171,-0.020883,-0.025521,0.102319
Days admitted,0.002694,-0.005122,0.251168,0.051901,0.197382,1.000000,0.199738,0.193763,0.029432,0.218131,0.004323,0.165975,0.158053,0.130913,0.293723,0.288457,0.117468
DiagnosisScore,0.002354,-0.009017,0.354463,0.236899,0.417789,0.199738,1.000000,0.357156,0.215654,0.494135,-0.000749,0.080818,0.174348,0.181086,0.061080,0.059372,0.144586
ProcedureScore,-0.000090,-0.013178,0.642231,0.304488,0.596615,0.193763,0.357156,1.000000,0.278149,0.694206,-0.001436,0.038063,0.267821,0.219568,-0.002370,-0.008703,0.081681
PhysicianScore,0.002024,-0.003733,0.175243,0.042899,0.133632,0.029432,0.215654,0.278149,1.000000,0.149155,-0.004487,0.015064,0.061308,0.039053,0.033917,0.036117,-0.005085
NaN count,0.000256,-0.017846,0.655173,0.412207,0.838225,0.218131,0.494135,0.694206,0.149155,1.000000,-0.001415,0.046049,0.295453,0.310721,-0.026233,-0.032430,0.127660


In [46]:
merged_df.columns

Index(['BeneID', 'Provider', 'InscClaimAmtReimbursed', 'ClmAdmitDiagnosisCode',
       'DiagnosisGroupCode', 'Days admitted', 'DiagnosisScore',
       'ProcedureScore', 'PhysicianScore', 'NaN count', 'Gender',
       'RenalDiseaseIndicator', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'DiseaseScore'],
      dtype='object')

In [47]:
#columns = []
#
#merged_df = merged_df.drop(columns = drop_cols, axis = 1)
#merged_df.info()
#merged_df.drop(['NaN count'],axis=1, inplace=True)
#merged_df_test.drop(['NaN count'],axis=1, inplace=True)

In [48]:
#merged_df_test.drop(['ClaimID','Provider','DeductibleAmtPaid','NaN count'],axis=1, inplace=True)

In [49]:
cor=merged_df.corr()
cor

,BeneID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DiagnosisGroupCode,Days admitted,DiagnosisScore,ProcedureScore,PhysicianScore,NaN count,Gender,RenalDiseaseIndicator,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,DiseaseScore
BeneID,1.000000,-0.000435,-0.000581,0.001221,0.000155,0.002694,0.002354,-0.000090,0.002024,0.000256,0.001393,0.004922,-0.001066,-0.000296,0.010718,0.008163,0.002141
Provider,-0.000435,1.000000,-0.011628,-0.008430,-0.016193,-0.005122,-0.009017,-0.013178,-0.003733,-0.017846,0.006047,0.001950,-0.009574,-0.013062,-0.004277,-0.006280,-0.003708
InscClaimAmtReimbursed,-0.000581,-0.011628,1.000000,0.279050,0.558359,0.251168,0.354463,0.642231,0.175243,0.655173,-0.001805,0.060811,0.383371,0.224641,0.066564,0.032133,0.092002
ClmAdmitDiagnosisCode,0.001221,-0.008430,0.279050,1.000000,0.344000,0.051901,0.236899,0.304488,0.042899,0.412207,-0.000896,0.028558,0.134413,0.145173,-0.015894,-0.014766,0.092864
DiagnosisGroupCode,0.000155,-0.016193,0.558359,0.344000,1.000000,0.197382,0.417789,0.596615,0.133632,0.838225,-0.000846,0.039226,0.248186,0.269171,-0.020883,-0.025521,0.102319
Days admitted,0.002694,-0.005122,0.251168,0.051901,0.197382,1.000000,0.199738,0.193763,0.029432,0.218131,0.004323,0.165975,0.158053,0.130913,0.293723,0.288457,0.117468
DiagnosisScore,0.002354,-0.009017,0.354463,0.236899,0.417789,0.199738,1.000000,0.357156,0.215654,0.494135,-0.000749,0.080818,0.174348,0.181086,0.061080,0.059372,0.144586
ProcedureScore,-0.000090,-0.013178,0.642231,0.304488,0.596615,0.193763,0.357156,1.000000,0.278149,0.694206,-0.001436,0.038063,0.267821,0.219568,-0.002370,-0.008703,0.081681
PhysicianScore,0.002024,-0.003733,0.175243,0.042899,0.133632,0.029432,0.215654,0.278149,1.000000,0.149155,-0.004487,0.015064,0.061308,0.039053,0.033917,0.036117,-0.005085
NaN count,0.000256,-0.017846,0.655173,0.412207,0.838225,0.218131,0.494135,0.694206,0.149155,1.000000,-0.001415,0.046049,0.295453,0.310721,-0.026233,-0.032430,0.127660


In [50]:
merged_df = merged_df.fillna(-1)

#Testing dataset
merged_df_test = merged_df_test.fillna(-1)

In [51]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538118 entries, 0 to 538117
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   BeneID                    538118 non-null  object 
 1   Provider                  538118 non-null  object 
 2   InscClaimAmtReimbursed    538118 non-null  int64  
 3   ClmAdmitDiagnosisCode     538118 non-null  float32
 4   DiagnosisGroupCode        538118 non-null  float32
 5   Days admitted             538118 non-null  int64  
 6   DiagnosisScore            538118 non-null  int64  
 7   ProcedureScore            538118 non-null  int64  
 8   PhysicianScore            538118 non-null  int64  
 9   NaN count                 538118 non-null  int64  
 10  Gender                    538118 non-null  int64  
 11  RenalDiseaseIndicator     538118 non-null  int64  
 12  IPAnnualReimbursementAmt  538118 non-null  int64  
 13  IPAnnualDeductibleAmt     538118 non-null  i

In [52]:
merged_df[['BeneID','Provider','ClmAdmitDiagnosisCode','DiagnosisGroupCode']] = merged_df[['BeneID','Provider','ClmAdmitDiagnosisCode','DiagnosisGroupCode']].astype('int64')


merged_df_test[['BeneID','Provider','ClmAdmitDiagnosisCode','DiagnosisGroupCode']] = merged_df_test[['BeneID','Provider','ClmAdmitDiagnosisCode','DiagnosisGroupCode']].astype('int64')

In [53]:
merged_df.columns

Index(['BeneID', 'Provider', 'InscClaimAmtReimbursed', 'ClmAdmitDiagnosisCode',
       'DiagnosisGroupCode', 'Days admitted', 'DiagnosisScore',
       'ProcedureScore', 'PhysicianScore', 'NaN count', 'Gender',
       'RenalDiseaseIndicator', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'DiseaseScore'],
      dtype='object')

In [54]:
merged_df_test.columns

Index(['BeneID', 'Provider', 'InscClaimAmtReimbursed', 'ClmAdmitDiagnosisCode',
       'DiagnosisGroupCode', 'Days admitted', 'DiagnosisScore',
       'ProcedureScore', 'PhysicianScore', 'NaN count', 'Gender',
       'RenalDiseaseIndicator', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'DiseaseScore'],
      dtype='object')

In [55]:
# Configuring the training and testing dataset
X_train = merged_df.drop(columns=['InscClaimAmtReimbursed'])
X_test = merged_df_test.drop(columns=['InscClaimAmtReimbursed'])
y_train = merged_df['InscClaimAmtReimbursed']
y_test = merged_df_test['InscClaimAmtReimbursed']

In [56]:
# Step 3: Train the Random Forest model on the training data
model = RandomForestRegressor(n_estimators=25, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=25, random_state=42)

In [57]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 8298576.845525849
R2 Score: 0.8421929171116886


In [58]:
import pickle
pickle.dump(model, open('model.pkl','wb'))

In [59]:
from sklearn.linear_model import LinearRegression

# Step 3: Train the Linear Regression model on the training data
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

LinearRegression()

In [60]:
y_pred = linear_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 24291179.56778091
R2 Score: 0.5380749905841531


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

# List of models to apply
models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge Regression', Ridge()),
    ('Lasso Regression', Lasso()),
    ('ElasticNet Regression', ElasticNet()),
    #('SVR', SVR()),
    ('Gradient Boosting Regressor', GradientBoostingRegressor()),
    ('XGBoost Regressor', XGBRegressor()),
    ('LightGBM Regressor', LGBMRegressor()),
    ('CatBoost Regressor', CatBoostRegressor(verbose=0)),
]

# Iterate over each model
for model_name, model in models:
    print(f"Training {model_name}...")
    # Training the model
    model.fit(X_train, y_train)
    
    # Predicting on the test set
    y_pred = model.predict(X_test)
    
    # Evaluating the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Printing the evaluation metrics
    print(f"{model_name} - Mean Squared Error: {mse}")
    print(f"{model_name} - R-squared (R2) Score: {r2}")
    print()

Training Linear Regression...
Linear Regression - Mean Squared Error: 24291179.56778091
Linear Regression - R-squared (R2) Score: 0.5380749905841531

Training Ridge Regression...
Ridge Regression - Mean Squared Error: 24291156.51175192
Ridge Regression - R-squared (R2) Score: 0.5380754290213389

Training Lasso Regression...
Lasso Regression - Mean Squared Error: 24290056.34827648
Lasso Regression - R-squared (R2) Score: 0.5380963499083766

Training ElasticNet Regression...
ElasticNet Regression - Mean Squared Error: 25784159.32550767
ElasticNet Regression - R-squared (R2) Score: 0.5096842454282313

Training Gradient Boosting Regressor...
Gradient Boosting Regressor - Mean Squared Error: 8541188.783937328
Gradient Boosting Regressor - R-squared (R2) Score: 0.8375793691519279

Training XGBoost Regressor...
XGBoost Regressor - Mean Squared Error: 7996499.422857314
XGBoost Regressor - R-squared (R2) Score: 0.8479372703622634

Training LightGBM Regressor...
[LightGBM] [Warning] Found whites

In [62]:
chosen_model = CatBoostRegressor(verbose=0)
chosen_model.fit(X_train, y_train)

In [63]:
# Predicting on the test set
y_pred = chosen_model.predict(X_test)

In [64]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R2 Score:", r2)

Mean Squared Error: 7388507.500853614
R2 Score: 0.8594989433354782


In [65]:
pickle.dump(model, open('model_final.pkl','wb'))